# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/3C%E5%AE%B6%E9%9B%BB/"  #3C家電

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [3]:
browser = webdriver.Chrome(executable_path='C:\\Users\\Rocky\\Downloads\\chromedriver.exe')
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    # Scroll down to the bottom.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load the page.
    time.sleep(2)

    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        break
    last_height = new_height

### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [4]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [5]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] YouTube推「輔助裁切」新工具！可直接刪除影音有版權爭議段落　不需下架刪除

▲Youtube讓創作者更容易發現版權問題。（圖／翻攝自pixabay）
記者陳心怡／外電報導
YouTube創作者有時可能會不慎誤用到別人的音樂或影片，一旦遭人檢舉或聲明有版權問題，創作者可能得撤下影片再重新編輯上傳。對此，YouTube日前推出更新功能，帶來一個名為「輔助裁切」的新版權工具，可以用來刪除影片中，受版權保護的他人影片或音樂，而無需整個刪除下架。
使用「輔助裁切」時，該功能會自動識別並突顯受版權聲明影響的影片部分，創建者可以從此處立即刪除，輕鬆解決問題。
除了Assisted Trim之外，YouTube還為創作者引入了其他幾種新的版權工具。其中之一是YouTube Studio中的新介面，使創作者除了能更清楚知道哪些影片的哪些部份侵權，也可以看到是誰宣稱對於內容擁有版權。
至於影片內容，音樂部分可以將侵權的部份靜音或移除；而如果是影片的話，則可以用剪輯功能將侵權的部份剪掉。
「輔助裁切」新版權工具可以讓創作者提早發現問題，並避開爭議。
其他人也看了這些～
►ETtoday 8歲歡慶！優惠好康都在這裡～～
----------------------------------------------------------------------

[1] OPPO Reno3亮相　首款雙模5G手機、搭影片超級雙防手震

▲OPPO Reno3。（圖／OPPO提供）
記者邱倢芯／綜合報導
OPPO正式推出了出首款支援雙模5G的智慧型手機Reno3 Pro和Reno3，以及全新的OPPO Enco Free真無線藍牙耳機。OPPO認為，作為Reno系列的第三代產品，Reno3系列在訊號、續航、散熱方面都進行了深度優化，全系列採用雙模5G晶片，以5G影像手機為基礎，打造出更全方位的5G智慧型手機。
值得注意的是，OPPO Reno3系列除了繼承Reno2在影片拍攝上的傑出表現，更推出了「影片超級雙防手震」功能，不管是日常或是運動狀態都可以幫助使用者拍出清晰又穩定的影片。
OPPO Reno3 Pro提供日出印象、藍色星夜、月夜